In [82]:
import pandas as pd
import math
import csv

hh_df = pd.read_csv('University/Summer 2020/TMG/GitHub_PORPOS/PORPOS-TMG/Data/SMTO_2015/SMTO_2015_Households.csv')
ps_df = pd.read_csv('University/Summer 2020/TMG/GitHub_PORPOS/PORPOS-TMG/Data/SMTO_2015/SMTO_2015_Respondents.csv')
campus_info = pd.read_csv('Campus_Info.csv')

# Load relevant columns
df = ps_df[['pscampusattend', 'personstatusgrad', 'personstatustime','psmainmodefalltypical', 'pscmpgender','psdrivinglicenseownerflag','psworknumhoursperweek','psborrowcar']]
df = df.join(hh_df[['HmTTS2006', 'hhlivingsituation', 'hhcarnumber','hhnumyoungerthan18','hhnumolderorequalto18','hhincomelevel']])

#df = ps_df[['pscampusattend','psmainmodefalltypical']]
#df = df.join(hh_df[['HmTTS2006','hhincomelevel']])

df = df.rename(columns={'HmTTS2006': 'HomeZone', 'pscampusattend': 'Campus', 'hhlivingsituation': 'Family', 'psmainmodefalltypical': 'Mode', 
                       'personstatusgrad': 'Level', 'personstatustime': 'Status', 'psuniversityinvolvednumyears': 'Years', 'hhcarnumber': 'Cars',
                       'hhincomelevel': 'Income', 'pscmpgender': 'Gender', 'psdrivinglicenseownerflag': 'Licence', 'psworknumhoursperweek': 'Work',
                       'hhnumyoungerthan18': 'Children', 'hhnumolderorequalto18':'Adults', 'psborrowcar': 'Car_Avail'})

#df = df.rename(columns={'HmTTS2006': 'HomeZone', 'pscampusattend': 'Campus', 'psmainmodefalltypical': 'Mode', 'hhincomelevel': 'Income'})

mode_name_to_num = {"Car - Driver alone": "Auto", "Car - Driver with passenger(s)": "Auto", "Car - Passenger": "Auto", "Taxi": "Auto", 
                    "Transit Bus": "Transit", "Streetcar": "Transit", "Subway/RT": "Transit", "GO Bus": "Transit", "GO Train": "Transit", 
                    "Walk": "Active", "Bicycle": "Active"}
print(df.shape)

df.replace({'Mode': mode_name_to_num}, inplace=True)
df = df[(df['Mode'] == "Auto") | (df['Mode'] == "Transit") | (df['Mode'] == "Active")]
print(df.shape)

df['Work'].fillna('Unknown', inplace=True) # Fill 11,000 Work NaNs with "Unknowns"
df['Car_Avail'].fillna(0, inplace = True)
df = df.dropna() # Remove rows with missing data
print(df.shape)


# Convert Campus column to numerical column
df['Campus'] = df["Campus"].apply(lambda x: campus_info.Zone[campus_info['School'] == x].tolist()[0])
print(df.shape)

# Convert columns to numerical
df['HomeZone'] = pd.to_numeric(df['HomeZone'], downcast='signed')
df['Car_Avail'] = pd.to_numeric(df['Car_Avail'], downcast='signed')

print(df.shape)
df.head()


(15226, 14)
(15066, 14)
(14839, 14)
(14839, 14)
(14839, 14)


,Campus,Level,Status,Mode,Gender,Licence,Work,Car_Avail,HomeZone,Family,Cars,Children,Adults,Income
0,566,UG,FT,Transit,Female,0,Unknown,0,261,Live with family/parents,1,3,4,Unknown
1,69,Grad,FT,Active,Female,1,Unknown,0,71,Live with partner,0,0,2,"$ 90,000 - 119,999"
2,69,UG,FT,Transit,Female,1,Unknown,1,3714,Live with family/parents,1,0,4,Unknown
3,69,UG,FT,Active,Male,1,Unknown,1,74,Live with roommates,0,0,4,Unknown
4,69,Grad,FT,Active,Male,1,Unknown,0,71,Live with partner,0,0,2,"$ 30,000 - 59,999"


In [62]:
df['Income'].value_counts()

Unknown                6910
Less than $ 30,000     2133
I donâ€™t know         1929
$ 30,000 - 59,999      1315
$ 60,000 - 89,999       944
$ 90,000 - 119,999      657
$ 120,000 - 149,999     369
$ 150,000 - 179,999     236
$ 240,000 +             155
$ 180,000 - 209,999     129
$ 210,000 - 239,999      62
Name: Income, dtype: int64

In [83]:
# Column Transformation functions

def inc_transform(x):
    if x == "Less than $ 30,000":
        return "Low"
    elif x == "$ 30,000 - 59,999":
        return "Low"
    elif x == "$ 60,000 - 89,999":
        return "Low"
    elif x == "$ 90,000 - 119,999":
        return "High"
    elif x == "$ 120,000 - 149,999":
        return "High"
    elif x == "$ 150,000 - 179,999":
        return "High"
    elif x == "$ 180,000 - 209,999":
        return "High"
    elif x == "$ 210,000 - 239,999":
        return "High"
    elif x == "$ 240,000 +":
        return "High"
    else:
        return "Unknown"
   
    
def work_transform(x):
    if x == "Yes, I work > 40 hours per week":
        return "FT"
    elif x == "Yes, I work 31-40 hours per week":
        return "FT"
    elif x == "Yes, I work part time (21-30 hours per week)":
        return "PT"
    elif x == "Yes, I work part time (11-20 hours per week)":
        return "PT"
    elif x == "Yes, I work part time (<10 hours per week)":
        return "PT"
    else:
        return "Other"
    
def fam_transform(x):
    if x == "Live with family/parents":
        return 1
    else:
        return 0

def car_transform(x):
    if x == 0:
        return 0
    else:
        return 1

In [86]:
df['Work'].value_counts()

Other    12990
PT        1591
FT         258
Name: Work, dtype: int64

In [85]:
df['Work'] = df["Work"].apply(lambda x: work_transform(x))
df['Family'] = df["Family"].apply(lambda x: fam_transform(x))
df['Cars'] = df["Cars"].apply(lambda x: car_transform(x))
df['Income'] = df["Income"].apply(lambda x: inc_transform(x))

In [65]:
df.head()

,Campus,Level,Status,Mode,Gender,Licence,Work,Car_Avail,HomeZone,Family,Cars,Children,Adults,Income
0,566,UG,FT,Transit,Female,0,Other,0,261,0,1,3,4,Unknown
1,69,Grad,FT,Active,Female,1,Other,0,71,0,0,0,2,High
2,69,UG,FT,Transit,Female,1,Other,1,3714,0,1,0,4,Unknown
3,69,UG,FT,Active,Male,1,Other,1,74,0,0,0,4,Unknown
4,69,Grad,FT,Active,Male,1,Other,0,71,0,0,0,2,Low


In [2]:
def row_to_segment(x):
    if x.Level == 'Other':
        return 0
    elif x.Level == 'UG':
        return 3 if x.Status == 'PT' else (not x.Family) + 1
    else:
        return 6 if x.Status == 'PT' else (4 + (not x.Family))

df['Segment'] = df.apply(row_to_segment, axis=1)

In [87]:
# Dataframe with walk distances
df_path = pd.read_csv('University/Summer 2020/TMG/GitHub_PORPOS/LoS/Walk_Distances.csv')
origins = list(set(list(df_path['Origin'])))
dists = list(df_path['Data'])

# Dataframe with AutoTravelTimes
df_att = pd.read_csv('University/Summer 2020/TMG/GitHub_PORPOS/LoS/Auto_Travel_Times.csv')
AutoTravelTimes = list(df_att['Data'])

# Dataframe with TransitTravelTimes
df_ttt = pd.read_csv('University/Summer 2020/TMG/GitHub_PORPOS/LoS/Transit_Travel_Times.csv')
TransitTravelTimes = list(df_ttt['Data'])

# Dataframe with TransitTravelTimes
df_ac = pd.read_csv('University/Summer 2020/TMG/GitHub_PORPOS/LoS/Auto_Cost.csv')
AutoCosts = list(df_ac['Data'])

# Dataframe with TransitTravelTimes
df_tc = pd.read_csv('University/Summer 2020/TMG/GitHub_PORPOS/LoS/Transit_Cost.csv')
TransitCosts = list(df_tc['Data'])

not_found = set()

# Function for distance/AutoTravelTime/TransitTravelTime lookup
def find_value(origin, destination, mode_num):
    # mode_num: 0 - to find Auto Travel Time
    #           1 - to find Transit Travel Time
    #           2 - to find Walking Distance
    #           3 - to find Auto Cost
    #           4 - to find Transit Cost
    try:
        i = origins.index(origin)
    except ValueError:
        not_found.add(origin)
        return 0
    try:
        j = origins.index(destination)
    except ValueError:
        not_found.add(destination)
        return 0
    
    if mode_num == 0:
        return AutoTravelTimes[i*2392 + j]
    elif mode_num == 1:
        return TransitTravelTimes[i*2392 + j]
    elif mode_num == 2:
        return dists[i*2392 + j] * 15/1000 # This was changed from walking distance to walking time
    elif mode_num == 3:
        return AutoCosts[i*2392 + j]
    elif mode_num == 4:
        return TransitCosts[i*2392 + j]
    else:
        print("ERROR: Enter correct mode_num!")
        return 0

'''
# List of campus' TTS zones from Joven's MOE data
campus_zones = list(campus_info['Zone'])
school_codes = list(campus_info['Code'])

# Load distances into dataframe
for i in range(len(campus_zones)):
    df["Dist." + school_codes[i]] = df['HomeZone'].apply(lambda x: find_distance(x, campus_zones[i]))
print("# of zones not found:", len(not_found))
'''

# Add columns for Distance, Transit Travel Time, Auto Travel Time, Auto Cost, Transit Cost
df['Time.Auto'] = df.apply(lambda x: find_value(x.HomeZone, x.Campus, 0), axis=1)
df['Time.Transit'] = df.apply(lambda x: find_value(x.HomeZone, x.Campus, 1), axis=1)
df['Time.Active'] = df.apply(lambda x: find_value(x.HomeZone, x.Campus, 2), axis=1)
#df['Cost.Auto'] = df.apply(lambda x: find_value(x.HomeZone, x.CampusZone, 3), axis=1)
#df['Cost.Transit'] = df.apply(lambda x: find_value(x.HomeZone, x.CampusZone, 4), axis=1)

df.head()

,Campus,Level,Status,Mode,Gender,Licence,Work,Car_Avail,HomeZone,Family,Cars,Children,Adults,Income,Time.Auto,Time.Transit,Time.Active
0,566,UG,FT,Transit,Female,0,Other,0,261,1,1,3,4,Unknown,17.474220,75.468478,223.214700
1,69,Grad,FT,Active,Female,1,Other,0,71,0,0,0,2,High,2.924953,24.128386,16.985265
2,69,UG,FT,Transit,Female,1,Other,1,3714,1,1,0,4,Unknown,50.171880,155.551170,349.788450
3,69,UG,FT,Active,Male,1,Other,1,74,0,0,0,4,Unknown,1.049121,16.675728,10.491210
4,69,Grad,FT,Active,Male,1,Other,0,71,0,0,0,2,Low,2.924953,24.128386,16.985265


In [26]:
# Load enrollment data
enrollment_df = pd.read_csv('../../Data/Enrolment/Joven_Enrollment.csv').set_index('School')
def get_log_enrollment(level, school):
    return math.log1p(enrollment_df.loc[school][level]) # Level: UG, Grad, Total
    
for level in ('Total', 'UG', 'Grad'):
    for i in range(len(campus_zones)):
        df[level + "." + school_codes[i]] = get_log_enrollment(level, school_codes[i])

FileNotFoundError: [Errno 2] File b'../../Data/Enrolment/Joven_Enrollment.csv' does not exist: b'../../Data/Enrolment/Joven_Enrollment.csv'

In [5]:
df = df[(df['Dist.SG'] > 0) | (df['Dist.SC'] > 0)]
df = df.drop(columns=['Family', 'Level', 'Status', 'Years', 'HomeZone'])
for i in range(7):
    for campus, count in df[df['Segment'] == i]['Campus'].value_counts().iteritems():
        df["S" + str(i) + "." + campus] = count

In [16]:
df

,Campus,Mode,HomeZone,Income,Time.Auto,Time.Transit,Time.Active
0,566,Transit,261,Unknown,17.474220,75.468478,223.214700
1,69,Active,71,"$ 90,000 - 119,999",2.924953,24.128386,16.985265
2,69,Transit,3714,Unknown,50.171880,155.551170,349.788450
3,69,Active,74,Unknown,1.049121,16.675728,10.491210
4,69,Active,71,"$ 30,000 - 59,999",2.924953,24.128386,16.985265
5,69,Active,72,Unknown,3.068723,25.742024,23.927640
6,566,Transit,600,Unknown,14.542720,53.745301,172.493400
7,566,Auto,3420,"$ 30,000 - 59,999",60.657420,213.667410,694.200900
8,69,Active,113,Unknown,15.759180,42.637287,93.900495
9,69,Transit,1031,Unknown,66.121790,117.968371,446.012700


In [88]:
# Remove columns we don't need anymore:
del df['Campus']
del df['HomeZone']

In [89]:
df.head()

,Level,Status,Mode,Gender,Licence,Work,Car_Avail,Family,Cars,Children,Adults,Income,Time.Auto,Time.Transit,Time.Active
0,UG,FT,Transit,Female,0,Other,0,1,1,3,4,Unknown,17.474220,75.468478,223.214700
1,Grad,FT,Active,Female,1,Other,0,0,0,0,2,High,2.924953,24.128386,16.985265
2,UG,FT,Transit,Female,1,Other,1,1,1,0,4,Unknown,50.171880,155.551170,349.788450
3,UG,FT,Active,Male,1,Other,1,0,0,0,4,Unknown,1.049121,16.675728,10.491210
4,Grad,FT,Active,Male,1,Other,0,0,0,0,2,Low,2.924953,24.128386,16.985265


In [90]:
# Dataframe to .csv file:
#df.to_csv('../../R_Logit_Models/Mode Choice Models/MChInput_2015_withIncome.csv')
df.to_csv('MChInput_2015_withColumns_2.1.csv', index = False)

In [93]:
df['Work'].value_counts()

Other    12990
PT        1591
FT         258
Name: Work, dtype: int64